# Drowsinness

The objective is to say if a driver is tired or not, to avoid car accidents. The idea of this file is to preprocess face images to generate a model which is able to predict drowsiness.

### Imports

In [ ]:
from google.colab import drive
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab import files
from datetime import datetime

### Load Data

Mount Google Drive

In [9]:
drive.mount('/content/drive', force_remount=True) #se utiliza para montar Google Drive dentro del entorno de Colab. Esto permite acceder y manipular los archivos almacenados en tu Google Drive directamente desde tu cuaderno de Colab

Mounted at /content/drive


Images Directory

In [ ]:
data_dir='/content/drive/Shareddrives/PBLayer/PBL5/01-Subjects/01-AdimenArtifiziala/Data/Processed/'

##Load and split data

In [10]:
train_set, validation_set = tf.keras.utils.image_dataset_from_directory(
        data_dir, #images directory
        image_size=(64, 64), #images size
        seed=1, # Especifica una semilla para la aleatorización. Esto se usa para garantizar que, si necesitas dividir el conjunto de datos en entrenamiento y validación de manera aleatoria, la división sea reproducible si se usa la misma semilla.
        validation_split=0.2, #the percentage of data for validation
        subset='both', #Esto significa que se creará un conjunto de datos que contendrá tanto los datos de entrenamiento como los de validación, dividiendo según el porcentaje especificado en validation_split
    )

Found 6797 files belonging to 2 classes.
Using 5438 files for training.
Using 1359 files for validation.


##Preprocessing

Scaling

In [ ]:
train_set = train_set.map(lambda x,y:(x/255,y)) #train normalization
validation_set = validation_set.map(lambda x,y:(x/255,y)) #validation normalization

##Model with Convolutional Layers and Drop Out

In [ ]:
ModeloCNN2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)), #agrega una capa de convolución con 32 filtros de tamaño 3x3, activada por la función de activación ReLU
    tf.keras.layers.MaxPooling2D(2, 2), #Agrega una capa de Max Pooling para reducir la dimensionalidad espacial de la salida de la capa convolucional.
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(), #Esta capa aplana la salida de la última capa de convolución (transformándola en un vector unidimensional) para prepararla para las capas densamente conectadas.
    tf.keras.layers.Dropout(0.5), #Agrega una capa de Dropout para ayudar a prevenir el sobreajuste, desactivando aleatoriamente el 50% de las neuronas durante el entrenamiento.
    tf.keras.layers.Dense(256, activation='relu'), #Una capa densa con 256 neuronas y activación ReLU.
    tf.keras.layers.Dense(1, activation='sigmoid') #Capa de salida con una sola neurona y activación sigmoide, comúnmente usada para problemas de clasificación binaria.
])

##Training the CNN Model with Drop Out

In [ ]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
ModeloCNN2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#optimizer='adam': Define el optimizador que se utilizará durante el entrenamiento del modelo. En este caso, se utiliza el optimizador Adam, que es un algoritmo de optimización popular y eficiente para entrenar redes neuronales.
#loss='binary_crossentropy': Establece la función de pérdida que se utilizará para evaluar qué tan bien está aprendiendo el modelo durante el entrenamiento. En este caso, se emplea la función de pérdida 'binary_crossentropy', que es comúnmente usada para problemas de clasificación binaria.
#metrics=['accuracy']: Especifica las métricas que se utilizarán para evaluar el rendimiento del modelo durante y después del entrenamiento.

ModeloCNN2.fit(train_set, epochs=10, batch_size=32, validation_data=validation_set, callbacks=[tensorboard_callback])
#train_set: Este es el conjunto de datos de entrenamiento que se utiliza para ajustar los pesos del modelo.
#epochs=10: Indica la cantidad de veces que todo el conjunto de datos de entrenamiento se pasará hacia adelante y hacia atrás a través de la red neuronal durante el entrenamiento.
#batch_size=32: Especifica el número de muestras que se utilizan en cada iteración de entrenamiento. El modelo se actualiza después de cada lote. En este caso, se utiliza un tamaño de lote de 32.
#validation_data=validation_set: Aquí se proporciona el conjunto de datos de validación para evaluar el rendimiento del modelo en datos que no se utilizan en el entrenamiento. Esto ayuda a monitorear si el modelo está sobreajustando o generalizando bien.
#callbacks=[tensorboard_callback]: Los callbacks son funciones que se llaman durante el entrenamiento en diferentes etapas. En este caso, se utiliza tensorboard_callback, que probablemente sea un objeto que se utiliza para guardar información de entrenamiento para su visualización en TensorBoard, una herramienta de visualización de TensorFlow.

Epoch 1/10
170/170 [==============================] - 673s 3s/step - loss: 0.4745 - accuracy: 0.7685 - val_loss: 0.3317 - val_accuracy: 0.8506
Epoch 2/10
170/170 [==============================] - 51s 290ms/step - loss: 0.2949 - accuracy: 0.8733 - val_loss: 0.2436 - val_accuracy: 0.9021
Epoch 3/10
170/170 [==============================] - 48s 279ms/step - loss: 0.2302 - accuracy: 0.9036 - val_loss: 0.2159 - val_accuracy: 0.9161
Epoch 4/10
170/170 [==============================] - 49s 287ms/step - loss: 0.1938 - accuracy: 0.9193 - val_loss: 0.1838 - val_accuracy: 0.9205
Epoch 5/10
170/170 [==============================] - 48s 277ms/step - loss: 0.1778 - accuracy: 0.9226 - val_loss: 0.1655 - val_accuracy: 0.9294
Epoch 6/10
170/170 [==============================] - 49s 285ms/step - loss: 0.1672 - accuracy: 0.9272 - val_loss: 0.1799 - val_accuracy: 0.9242
Epoch 7/10
170/170 [==============================] - 50s 288ms/step - loss: 0.1419 - accuracy: 0.9402 - val_loss: 0.1736 - val_accu

##Save the model

In [ ]:
ModeloCNN2.save('/content/drive/Shareddrives/PBLayer/PBL5/01-Subjects/01-AdimenArtifiziala/Neka_IA_3.h5') #to save model --->.h5 format
ModeloCNN2.save_weights('/content/drive/Shareddrives/PBLayer/PBL5/01-Subjects/01-AdimenArtifiziala/Neka_pesosIA_3.h5') #Esto no guarda la arquitectura o la configuración del modelo, solo los valores numéricos de los pesos que han sido aprendidos durante el entrenamiento.
print('Saved model')

Terminamos Modelo CNN


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Pesos

 En una CNN, los pesos son matrices numéricas que representan la importancia y la relación entre las diferentes características (como bordes, texturas, etc.) presentes en las imágenes o datos de entrada.

Los pesos se utilizan en las capas de la red neuronal para realizar operaciones matemáticas en los datos de entrada y producir salidas que se acerquen lo más posible a las salidas deseadas. Estos pesos son ajustados iterativamente durante el entrenamiento para minimizar la diferencia entre las predicciones del modelo y las etiquetas reales de los datos de entrenamiento.